# Access and Archive Inundation from 1,000 Climate Realizations (No Obstruction)

The purpose of this notebook is to access, process, and archive inundation acrosss the 1,000 climate realizations and approximately 3,500 daily events that trigger the initiation event.

## Parameters and Imports

In [1]:
%matplotlib inline

In [2]:
import scipy.stats as sstats
from IPython.display import display, HTML
import os
import datetime as dt
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

In [3]:
IN_DIR = r'C:\Users\nmart\Dropbox\MyPapers\Flood_Risk_PRA\Flood_Risk_Model\PRA_EventTrees_no_Blockage\Results'

In [4]:
XLSX_NAME_ROOT = "R%04dto%04d_Flooding_Summary_All.xlsx"
REAL_INC = 50
START_REAL = 1
FINAL_REAL = 1000

Revised cost calculation in "Flood_Risk_PRA\Damage_Cost\NFIP_Cost_Estimator.xlsx"

In [5]:
ccA = -72769.0
ccB = 414681.0 
ccC = -678009.0 
ccD = 498224.0
ccE = 37110.0
MAX_COST = 750000.0

In [6]:
def costCalc( IDepth, ccA, ccB, ccC, ccD, ccE, MAX_COST ):
    """ Custom inundation damage cost curve calculation.
    
    Fourth degree polynomial with coefficients: A, B, C, D, E, and
    cost limited to MAX_COST.
    
    Args:
        IDepth (float): depth of inundation in meters
        ccA (float): A coefficient for x^4
        ccB (float): B coefficient for x^3
        ccC (float): C coefficient for x^2
        ccD (float): D coefficient for x^1
        ccE (float): E coefficient for x^0
        
    Returns:
        iCost (float): varies between 0.0 and MAX_COST
    """
    from math import pow
    # check for positive depth and extrapolation.
    if IDepth <= 0.0:
        return 0.0
    elif IDepth >= 2.75:
        return MAX_COST
    # end if
    # calculate the polynomial
    estCost = ( ( ccA * pow( IDepth, 4.0 ) ) + ( ccB * pow( IDepth, 3.0 ) ) + ( ccC * pow(IDepth, 2.0) ) +
                ( ccD * pow( IDepth, 1.0 ) ) + ccE )
    if estCost > MAX_COST:
        iCost = MAX_COST
    elif estCost <= 0.0:
        iCost = 0.0
    else:
        iCost = estCost
    # end if
    return iCost

## Process and Compile Simulation Results

In [7]:
RealNumList = list()
RealCostList = list()
SummaryDF_List = list()
EventDF_List = list()

In [8]:
rnCnt = 1

In [9]:
startInd = START_REAL
endInd = REAL_INC
while (startInd <= FINAL_REAL):
    # get the name for this spreadsheet
    InFiler = os.path.normpath( os.path.join( IN_DIR, XLSX_NAME_ROOT % ( startInd, endInd ) ) )
    if not os.path.isfile( InFiler ):
        print("File %s does not exist!!!" % InFiler)
        break
    # end if
    print("Working on MC collation run %d" % rnCnt)
    curSumDF = pd.read_excel( InFiler, sheet_name="Summary", index_col=0, header=0, parse_dates=[3,], )
    SummaryDF_List.append( curSumDF )
    # now go through the summary sheet and read in the individual flooding events.
    preReal = 0
    totRealCost = 0.0
    for indx1, row in curSumDF.iterrows():
        cReal = int( row["Realization"] )
        # determine if can assign total cost and set current realization
        if cReal == preReal:
            curReal = preReal
        else:
            curReal = cReal
            if preReal != 0:
                RealNumList.append( preReal )
                RealCostList.append( totRealCost )
            # end if
            preReal = cReal
            totRealCost = 0.0
        # end if
        # get the flood number so can read in the "right" sheet
        cFl = int( row["Flood Num."] )
        sName = "Inun_R%04d_Fl%02d" % ( curReal, cFl )
        # read in this sheet and calculate total flood cost
        cFlDF = pd.read_excel( InFiler, sheet_name=sName, header=0, index_col=0, )
        # add climate realization and house index column
        cFlDF["Realization"] = curReal
        cFlDF["House_Ind"] = cFlDF.index
        # add cost estimate column
        cFlDF["Cost_Estimate"] = cFlDF.apply( lambda row: costCalc( row["FloodDepth_m"], ccA, ccB, ccC, ccD, ccE, MAX_COST ),
                                              axis=1, )
        # add this DataFrame to the tracking list
        EventDF_List.append( cFlDF )
        # calculate cost for this event for summary tracking
        totFloodCost = cFlDF["Cost_Estimate"].sum()
        totRealCost += totFloodCost
    # end summary iteration
    # increment counters
    startInd += REAL_INC
    endInd += REAL_INC
    rnCnt += 1
# end while loop
startInd, endInd

Working on MC collation run 1
Working on MC collation run 2
Working on MC collation run 3
Working on MC collation run 4
Working on MC collation run 5
Working on MC collation run 6
Working on MC collation run 7
Working on MC collation run 8
Working on MC collation run 9
Working on MC collation run 10
Working on MC collation run 11
Working on MC collation run 12
Working on MC collation run 13
Working on MC collation run 14
Working on MC collation run 15
Working on MC collation run 16
Working on MC collation run 17
Working on MC collation run 18
Working on MC collation run 19
Working on MC collation run 20


(1001, 1050)

## Output Processed and Collated

In [10]:
FullHouseRealDF = pd.concat( EventDF_List, ignore_index=True, )

In [11]:
OutFiler = os.path.normpath( os.path.join( IN_DIR, "FullHouseRealDF_noObs.pkl" ) )
FullHouseRealDF.to_pickle( OutFiler, compression='zip', protocol=-1, )

In [12]:
display( HTML( FullHouseRealDF.head().to_html() ) )

,Row,Column,Topo_m,FloorEl_m,FloorHeight_m,WaterDepth_m,FloodDepth_m,Realization,House_Ind,Cost_Estimate
0,114,27,109.349998,109.633003,0.283002,0.0,0.0,1,1,0.0
1,119,27,109.099998,109.383003,0.283002,0.0,0.0,1,2,0.0
2,124,27,108.849998,109.133003,0.283002,0.0,0.0,1,3,0.0
3,129,27,108.599998,108.883003,0.283002,0.0,0.0,1,4,0.0
4,134,27,108.349998,108.633003,0.283002,0.0,0.0,1,5,0.0


In [13]:
display( HTML( FullHouseRealDF.tail().to_html() ) )

,Row,Column,Topo_m,FloorEl_m,FloorHeight_m,WaterDepth_m,FloodDepth_m,Realization,House_Ind,Cost_Estimate
157647,144,44,107.849998,108.133003,0.283002,0.0,0.0,1000,40,0.0
157648,149,44,107.599998,107.883003,0.283002,0.0,0.0,1000,41,0.0
157649,154,44,107.349998,107.633003,0.283002,0.0,0.0,1000,42,0.0
157650,159,44,107.099998,107.383003,0.283002,0.0,0.0,1000,43,0.0
157651,164,44,106.849998,107.133003,0.283002,0.0,0.0,1000,44,0.0


In [14]:
display( HTML( FullHouseRealDF.describe().to_html() ) )

,Row,Column,Topo_m,FloorEl_m,FloorHeight_m,WaterDepth_m,FloodDepth_m,Realization,House_Ind,Cost_Estimate
count,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000,157652.000000
mean,139.000000,35.500000,104.349998,105.716503,1.366501,0.663179,0.163154,500.158805,22.500000,37403.026405
std,15.811438,6.800757,3.832440,2.781236,1.083503,1.321107,0.639912,288.899436,12.698465,140624.147643
min,114.000000,27.000000,99.349998,101.800003,0.283002,0.000000,0.000000,1.000000,1.000000,0.000000
25%,124.000000,30.000000,100.599998,103.050003,0.283002,0.000000,0.000000,252.000000,11.750000,0.000000
50%,139.000000,35.500000,104.349998,105.716503,1.366501,0.000000,0.000000,496.000000,22.500000,0.000000
75%,154.000000,41.000000,108.099998,108.383003,2.450001,0.853021,0.000000,751.000000,33.250000,0.000000
max,164.000000,44.000000,109.349998,109.633003,2.450001,8.470463,6.020461,1000.000000,44.000000,750000.000000


In [15]:
SumListDF = pd.concat( SummaryDF_List, ignore_index=True )

In [16]:
DataDict = { "Climate Realization" : np.array( RealNumList, dtype=np.int32 ),
             "Total Cost" : np.array( RealCostList, dtype=np.float32 ), }
OverviewDF = pd.DataFrame( data=DataDict )

In [17]:
OutFiler = os.path.normpath( os.path.join( IN_DIR, "Summary_CR1000_NoObs.xlsx" ) )

In [18]:
writer = pd.ExcelWriter( OutFiler )
workbook  = writer.book
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format': '#,##0.000'})
format3 = workbook.add_format({'num_format': '#,##0'})
cLabel = "Overview"
OverviewDF.to_excel( writer, sheet_name=cLabel, )
# adjust columns
writer.sheets[cLabel].set_column( 0, 0, 8 )
for column in OverviewDF:
    column_width = max(OverviewDF[column].astype(str).map(len).max()+6, len(column)+6)
    col_idx = OverviewDF.columns.get_loc(column)
    if column in ["Date",]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, )
    elif column in ["Climate Realization", "Flood Num."]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format3)
    else:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
# end for
cLabel = "Summary"
SumListDF.to_excel( writer, sheet_name=cLabel, )
# adjust columns
writer.sheets[cLabel].set_column( 0, 0, 18 )
for column in SumListDF:
    column_width = max(SumListDF[column].astype(str).map(len).max()+6, len(column)+6)
    col_idx = SumListDF.columns.get_loc(column)
    if column in ["Date",]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, )
    elif column in ["Realization", "Flood Num."]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format3)
    else:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format2)
# end for
writer.close()

In [19]:
display( HTML( OverviewDF.describe().to_html() ) )

,Climate Realization,Total Cost
count,979.000000,9.790000e+02
mean,499.775281,5.881578e+06
std,288.869970,8.196827e+06
min,1.000000,0.000000e+00
25%,250.000000,0.000000e+00
50%,499.000000,1.275181e+06
75%,750.000000,1.041440e+07
max,999.000000,5.368721e+07


In [20]:
display( HTML( SumListDF.tail().to_html() ) )

,Realization,Flood Num.,Date,Precip_mm,Discharge_cms,Obstruction_Depth_m,Max_Water_Depth_m,Max_Flood_Depth_m,Max_U_mps,Max_V_mps
3578,999,2,2061-01-27,245.074814,186.921478,0,1.505995,0.00000,1.320052,10.773865
3579,1000,1,2028-01-05,245.558990,187.290756,0,1.523685,0.00000,1.325256,10.787840
3580,1000,2,2050-01-09,238.121597,181.618179,0,1.277775,0.00000,1.257311,10.577990
3581,1000,3,2056-01-05,266.281342,203.095947,0,2.200465,0.00000,1.433184,11.109060
3582,1000,4,2061-01-02,328.370911,250.452393,0,3.993352,1.54335,1.870786,12.223204


In [21]:
display( HTML( SumListDF.describe().to_html() ) )

,Realization,Flood Num.,Precip_mm,Discharge_cms,Obstruction_Depth_m,Max_Water_Depth_m,Max_Flood_Depth_m,Max_U_mps,Max_V_mps
count,3583.000000,3583.000000,3583.000000,3583.000000,3583.0,3583.000000,3583.000000,3583.000000,3583.000000
mean,500.158805,2.607870,278.532065,212.439716,0.0,2.475549,0.603988,1.574413,11.296490
std,288.938843,1.486216,58.823586,44.865449,0.0,1.577051,1.250510,0.574233,0.940780
min,1.000000,1.000000,236.024506,180.018692,0.0,1.210083,0.000000,1.237521,10.407026
25%,252.000000,1.000000,242.891487,185.256218,0.0,1.432162,0.000000,1.299554,10.700460
50%,496.000000,2.000000,251.877472,192.109940,0.0,1.741803,0.000000,1.370767,10.854691
75%,751.000000,3.000000,286.403793,218.443581,0.0,2.797889,0.347887,1.569699,11.436788
max,1000.000000,10.000000,656.915161,501.036987,0.0,8.470463,6.020461,8.151344,15.275147
